# 참고문헌 식별화 작업

In [21]:
# import urllib.request
import requests
import json
import pandas as pd

import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = [
'https://spreadsheets.google.com/feeds',
'https://www.googleapis.com/auth/drive',
]
json_file_name = './gothic-imprint-335303-332f23bde214.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

from tqdm.notebook import trange, tqdm
import time
import re
import regex

In [2]:
clientId = 'RQ3UYCdEX3UddczsC2lE'
clientSecret = 'd3B8n1N71z'
headers = {'X-Naver-Client-Id': clientId, 'X-Naver-Client-Secret': clientSecret}

### 0. 참고문헌 목록 가져오기

In [3]:
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/14p2K2Y0owZN_R3NpKLG7y5cfZDPnpANnJRsQvntSeaU/edit#gid=872153593'
# 스프레스시트 문서 가져오기 
doc = gc.open_by_url(spreadsheet_url)
# 시트 선택하기
worksheet = doc.worksheet('통합')

values = worksheet.get_all_values()
header, rows = values[0], values[1:]

refs = pd.DataFrame(rows, columns=header)
refs

,논문번호,정렬용,구분,기타구분,저자,편(編),번역자,연도,월일,제목(1),제목(2),출판,학술지_권(집),학술지_호,학위구분,기타정보
0,200,8813,?,,백로라,,,2011,,,「한국 현대 실험연극의 담론과 미학적 전통의 창조」,"한국연구재단(NRF), 2008년 선정 기초지원인문사회(단독연구)사업 결과 보고서",,,,
1,211,8929,?,,박명규,,,2006,12,「경성제국대학 총장과 식민지 대학像」,식민권력과 근대지식: 경성제국대학연구 Workshop,,,,,
2,211,8968,?,,장지영,,,2018,9.20.,「‘상아탑’의 토포스: 해방 후 문학에서의 학술 재현」,,《ICE(성균관대학교 한국학/동아시아학 연합 콜로키엄)》,,,,
3,211,8969,?,,장지영,,,2019,7.5.-6.,「통합과 이화: (국문)학과의 제도화 과정과 권위의 지정학적 분여(分與)」,,《제3회 인문한국학학술대회: 동아시아 한국학의 소통과 확산》,,,,
4,171,7973,?,,김형수,,,2019,4.5.,"｢신동엽의 고독한 길, 영성적 근대｣",,"따로, 다르게(신동엽 시인 50주기 기념자료집)",,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16444,377,16192,해외단행본,,韓雪野,,陶炳蔚 譯,1960,,,塔,上海文藝出版社,,,,
16445,377,16193,해외단행본,,李箕永譯,,陶炳蔚,1957,,,土地,作家出版社,,,,
16446,377,16194,해외단행본,,彭德懷,,,1981,,,彭德懷自述,人民出版社,,,,
16447,367,16195,해외단행본,,松原寛,,,1921,,,現代人の芸術,民衆文化協会出版部,,,,


In [4]:
refs.columns

Index(['논문번호', '정렬용', '구분', '기타구분', '저자', '편(編)', '번역자', '연도', '월일', '제목(1)',
       '제목(2)', '출판', '학술지_권(집)', '학술지_호', '학위구분', '기타정보'],
      dtype='object')

In [20]:
refs['구분'].value_counts()

국내단행본    6181
국내논문     5701
기본자료     1900
학위논문     1616
해외단행본     750
해외논문      261
기타         30
?          10
Name: 구분, dtype: int64

### 1. 국내/국외 단행본
네이버 책 검색 활용 : https://developers.naver.com/docs/serviceapi/search/book/book.md

In [14]:
def NAVER_ISBN(title, author, year):
    book_url = f"https://openapi.naver.com/v1/search/book_adv.json?start=1&d_titl={title}&d_auth={author}&d_dafr={year}0101&d_dato={year}1231"
    r = requests.get(book_url, headers=headers)
    time.sleep(0.1)
    req = json.loads(r.text)
    # print(req)
    if len(req['items'])!=0:
        isbn = req['items'][0]['isbn']
    else:
        isbn = None
    return isbn

In [7]:
refs[refs['구분'].str.contains('단행본')]

,논문번호,정렬용,구분,기타구분,저자,편(編),번역자,연도,월일,제목(1),제목(2),출판,학술지_권(집),학술지_호,학위구분,기타정보
4609,348,13129,국내단행본,,김재홍,,,1999,,「생명. 사랑. 자유의 시학」,『김재홍 비평집』,,,,,
4612,348,13133,국내단행본,,,대한성서공회 성경편집팀 편,,1982,,,『공동번역성서』,대한성서공회,,,,
4614,348,13135,국내단행본,,박이도,,,1988,,「신앙적인 아픔과 호소 그리고 사랑의 정신」,『깨어나소서 주여』,종로서적,,,,
4622,348,13145,국내단행본,,신은경 외,,,1996,,「여성성의 구현으로서 여성텍스트와 여성문체-김남조시를 중심으로」,『한국 페미니즘 시학』,동화서적,,,,
4624,348,13150,국내단행본,,오세영,,,2005,,「사랑의 플라토니즘과 구원」,『김남조 시전집』,국학자료원,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16444,377,16192,해외단행본,,韓雪野,,陶炳蔚 譯,1960,,,塔,上海文藝出版社,,,,
16445,377,16193,해외단행본,,李箕永譯,,陶炳蔚,1957,,,土地,作家出版社,,,,
16446,377,16194,해외단행본,,彭德懷,,,1981,,,彭德懷自述,人民出版社,,,,
16447,367,16195,해외단행본,,松原寛,,,1921,,,現代人の芸術,民衆文化協会出版部,,,,


In [8]:
for idx in tqdm(refs[refs['구분'].str.contains('단행본')].index):
    title = refs['제목(2)'][idx]
    author= refs['저자'][idx]
    year = refs['연도'][idx]
    isbn = NAVER_ISBN(title,author,year)
    refs.loc[idx,'isbn'] = isbn

  0%|          | 0/6931 [00:00<?, ?it/s]

In [9]:
refs['isbn'].dropna()

6035     8932008639 9788932008639
6037     2191000401 2002191000409
6039     2194002391 2002194002394
6040     8961950576 9788961950572
6065     304900004X 2003049000046
                   ...           
16385    4845918366 9784845918362
16401    4140019522 9784140019528
16411    4783709955 9784783709954
16413    4876162611 9784876162611
16422    4052047389 9784052047381
Name: isbn, Length: 2904, dtype: object

### 2. 학술자료

In [42]:
def NAVER_Aca(query):
    query = regex.sub('[^\p{Hangul}\p{Han}\d\w\s]','',query)
    academic_url = f'https://openapi.naver.com/v1/search/doc.json?query={query}'
    r = requests.get(academic_url, headers=headers)
    time.sleep(0.1)
    req = json.loads(r.text)
    # print(req)
    if len(req['items'])!=0:
        try:
            link = req['items'][0]['link']
            link = re.search('doc_id\=(\d+)',link).group(1)
        except AttributeError:
            link = req['items'][1]['link']
            link = re.search('doc_id\=(\d+)',link).group(1)
            print(link)
    else:
        link = None
    return link

In [46]:
for idx in tqdm(refs[refs['구분'].str.contains('논문')].index):
    title = refs['제목(1)'][idx]
    isbn = NAVER_Aca(title)
    refs.loc[idx,'isbn'] = isbn

  0%|          | 0/7578 [00:00<?, ?it/s]